# battle simulator

In [1]:
from __future__ import print_function, division
import json
import numpy as np
import math
import bisect
import sys
import time
try: input = raw_input
except NameError: pass
import random

In [ ]:
# minion instance
{
	"mid": 1,
	"uid": 1,
	"level": 60,
	"tot_xp": 150000,
	"xp_to_next_level": 6000,
    #"tot_stats_pt": 136,
    #"assigned_stats_pt": 136,
    #"free_stats_pt": 0,
    "type1":"flight",
    "type2":"longrange",
	"base_stats": [35, 20, 15, 23, 10, 17, 26],  # strength, endurance, dexterity, obedience, constituition, intelligence, wisdom
	#"stats_ceiling":[100, 100, 100, 100, 100, 100, 100],
	"cur_stats": [],  # base_stats * wellness_factor * luck
	"base_substats": [],  # attack, defense, critical, hit; f(base_stats)
	"cur_substats": [], # f(current_stats)
	"base_attributes_capacity": [],  # hit points, ability points, energy points; f(base_stats)
	"cur_attributes_capacity": [], # hit points, ability points, energy points; f(current_stats)
    "base_attributes": [1900, 490, 485], # keep track of depletion and replenishment, clipped by base_attributes_ceiling
    "cur_attributes": [], # base_attributes * luck, clipped by current_attributes_ceiling? problem: attibute ceiling is fluctuating
	"mp": 0, # meter points
    "mp_capacity": 200, 
    "mp_regen_per_second": 3, # mp regenerated per second
    "mp_gain_rate_damage_dealt": 0.2, #  mp += rate * damage dealt 
    "mp_gain_rate_damage_taken":0.3, # mp += rate * damage taken
    "states": [0,0,0],
    "stats_coeff": [[5, 3, 0.005, 0.015, 85, 50, 70], #coefficents
                    [60, 30, 0.20, 0.50, 500, 500, 500]], #initial substats and attributes at lvl 0
	"base_transition_matrix": [[0.5, 0.3, 0.2],
            [0.25, 0.5, 0.25],
            [0.2, 0.3, 0.5]],
	"cur_transition_matrix": [[0.5, 0.3, 0.2],
            [0.25, 0.5, 0.25],
            [0.2, 0.3, 0.5]], # f(base_transition_matrix, alpha)
	"wellness_goal": [1,1,1,1,1], # fast food vs grocery, gym vs movie, ...
    "wellness_factor": 1.25, # f(wellness_goal, transaction or user activities)
	"luck": [], # stats, attributes, skills, abilities
	"cp": 0, # f(cur_substats, cur_attributes)
	"skills": [1,2], # sid
	"abilities": []
}

In [ ]:
# skill instance
[{
		"id": 1,
		"name": "fast_move_1",
		"power": 20,
		"mpdelta": 0,
		"cast": 0, # cast time, how long it takes for the damage to be dealt
		"cd": 0.5, # cooldown
		"mtype": "f"
	},
	{
		"id": 2,
		"name": "super_move_1",
		"power": 800,
		"mpdelta": -80,
		"cast": 0,
		"cd": 10,
		"mtype": "s"
	}
]

### preprocessing minion and skill data

In [39]:
def generate_stats_coeff(minion): #randomize initial substats and attributes by 10%
    stats_coeff = np.array(minion['stats_coeff'])
    rand_mat = np.array([random.uniform(0.9,1.1) for _ in range(len(stats_coeff[1]))])
    stats_coeff[1] = np.multiply(stats_coeff[1], rand_mat)
    stats_coeff[1][0:2] = np.floor(stats_coeff[1][0:2]) + 1
    stats_coeff[1][4:7] = np.floor(stats_coeff[1][4:7]) + 1
    minion['stats_coeff'] = stats_coeff
    return minion


def generate_luck(minion, p = [0.2, 0.65, 0.15], magnitude = 0.3):
    # p: luck distribution, [bad, neutural, good]
    n_stats = len(minion['base_stats'])
    n_attributes = len(minion['base_attributes'])
    #n_skills = len(minion['smoves'])
    n_abilities = len(minion['abilities'])
    
    stats_luck = np.array([np.random.choice(1+np.arange(-1,2)*magnitude, p=p) for _ in range(n_stats)])
    attributes_luck = np.array([np.random.choice(1+np.arange(-1,2)*magnitude, p=p) for _ in range(n_attributes)])
    #skills_luck = np.array([np.random.choice(1-np.arange(-1,2)*magnitude, p=p) for _ in range(n_skills)])
    smove_luck = np.random.choice(1-np.arange(-1,2)*magnitude, p=p)
    abilities_luck = np.array([np.random.choice(1-np.arange(-1,2)*magnitude, p=p) for _ in range(n_abilities)])

    luck = np.array([stats_luck, attributes_luck, smove_luck, abilities_luck])
    #luck = np.array([stats_luck, attributes_luck, skills_luck, abilities_luck])
    
    minion['luck'] = luck
    return minion


def compute_current_status(minion): # f(base_stats, base_attributes, luck, wellness_factor)
    
    def clip_by_capacity(attr, attr_capacity):
        for i in range(len(attr)):
            attr[i] = min(attr[i], attr_capacity[i])
        return attr
    
    base_stats = np.array(minion['base_stats'])
    base_attributes = np.array(minion['base_attributes'])
    luck = np.array(minion['luck'])
    wellness_factor = minion['wellness_factor']
    stats_coeff = np.array(minion['stats_coeff'])
    
    # base_capacity = f(base_stats)
    base_capacity = np.floor(np.multiply(base_stats, stats_coeff[0]) + stats_coeff[1]) + 1
    base_substats = base_capacity[0:4]
    base_attributes_capacity = base_capacity[4:7]
    base_attributes = clip_by_capacity(base_attributes, base_attributes_capacity)
    
    # cur_stats = f(base_stats, luck, wellness_factor)
    cur_stats = np.floor((np.multiply(base_stats, luck[0]) * wellness_factor)) + 1
    
    # cur_capacity = f(cur_stats)
    cur_capacity = np.floor(np.multiply(cur_stats, stats_coeff[0]) + stats_coeff[1]) + 1
    cur_substats = cur_capacity[0:4]
    cur_attributes_capacity = cur_capacity[4:7]
    
    # cur_attributes = f(base_attributes, luck)
    cur_attributes = clip_by_capacity(np.floor(np.multiply(base_attributes, luck[1])) + 1, cur_attributes_capacity)
    
    # tbd: cp = f(cur_substats, cur_attributes, skill's DPS, ...)
    #cp = 
    
    # update
    minion['base_substats'] = base_substats
    minion['base_attributes_capacity'] = base_attributes_capacity
    minion['base_attributes'] = base_attributes
    minion['cur_stats'] = cur_stats
    minion['cur_substats'] = cur_substats
    minion['cur_attributes_capacity'] = cur_attributes_capacity
    minion['cur_attributes'] = cur_attributes
    #minion['cp'] = cp

    return minion

In [40]:
def load_minion_data(fn, update_luck = True, update_stats_coeff = True):
    #read minion data from json
    #(optional) generate luck, randomize stats coefficient
    #compute current status, index on mid, output dict
    
    minion_data = {}
    
    with open(fn,'r') as f:
        d = json.load(f)
            
    for m in d:
        if update_luck or len(m['luck']) == 0:
            m = generate_luck(m)
        if update_stats_coeff:
            m = generate_stats_coeff(m)
        m = compute_current_status(m)
        minion_data[m['id']] = m
    return minion_data


def load_data(fn):
    data = {}
    
    with open(fn,'r') as f:
        d = json.load(f)
    
    for l in d:
        data[l['id']] = l
    
    return data

In [41]:
# load data from json
minion_data = load_minion_data('miniondata_1.json')
skill_data = load_data('skilldata_1.json')

In [27]:
minion_data

{1: {'id': 1,
  'uid': 1,
  'level': 60,
  'tot_xp': 150000,
  'xp_to_next_level': 6000,
  'type1': 'flight',
  'type2': 'longrange',
  'base_stats': [35, 20, 15, 23, 10, 17, 26],
  'cur_stats': array([44., 26., 14., 29., 17., 22., 23.]),
  'base_substats': array([239.,  94.,   1.,   1.]),
  'cur_substats': array([284., 112.,   1.,   1.]),
  'base_attributes_capacity': array([1334., 1358., 2303.]),
  'cur_attributes_capacity': array([1929., 1608., 2093.]),
  'base_attributes': array([1334,  490,  485]),
  'cur_attributes': array([1335.,  491.,  486.]),
  'mp': 0,
  'mp_capacity': 200,
  'mp_regen_per_second': 3,
  'mp_gain_rate_damage_dealt': 0.2,
  'mp_gain_rate_damage_taken': 0.3,
  'states': [0, 0, 0],
  'stats_coeff': array([[5.00000000e+00, 3.00000000e+00, 5.00000000e-03, 1.50000000e-02,
          8.50000000e+01, 5.00000000e+01, 7.00000000e+01],
         [6.30000000e+01, 3.30000000e+01, 1.93605697e-01, 5.23670398e-01,
          4.83000000e+02, 5.07000000e+02, 4.82000000e+02]]),
  

In [28]:
skill_data

{1: {'id': 1,
  'name': 'fast_move_1',
  'power': 20,
  'mpdelta': 0,
  'cast': 0,
  'cd': 0.5,
  'mtype': 'f'},
 2: {'id': 2,
  'name': 'super_move_1',
  'power': 800,
  'mpdelta': -80,
  'cast': 0,
  'cd': 10,
  'mtype': 's'}}

### classes

In [44]:
class Skill:

    def __init__(self, **kwargs):
    # kwargs: id, name, power, mpdelta, cast, cd, mtype
        for k, v in kwargs.items():
            setattr(self, k, v)
        self.free_since = 0
    '''
    def load_from_dict(self, skill_dict, sid):
        this_skill = skill_dict[sid]
        self.id = this_skill['id']
        self.name = this_skill['name']
        self.power = this_skill['power']
        self.mpdelta = this_skill['mpdelta']
        self.cast = this_skill['cast']
        self.cd = this_skill['cd']
        self.mtype = this_skill['mtype']
     '''
    
#class fmove(Skill):
#    def __init__(self, sid, name, power, mpdelta, cast, cd, mtype):
#        super(fmove, self).__init__(sid, name, power, mpdelta, cast, cd, 'f')


#class smove(Skill):
#    def __init__(self, sid, name, power, mpdelta, cast, cd, mtype):
#        super(smove, self).__init__(sid, name, power, mpdelta, cast, cd, 's')


class BattleMinion:
    '''
    def __init__(self, **kwargs):
    # kwargs: mid, attack, defense, critical, hit, mp, mp_capacity, mp_rate1, mp_rate2, mp_rate3, luck, skills
        for k, v in kwargs.items():
            setattr(self, k, v)
    '''
    
    def __init__(self, minion_dict, skill_dict, mid):
        this_minion = minion_dict[mid]
        self.id = this_minion['id']
        self.attack = this_minion['cur_substats'][0]
        self.defense = this_minion['cur_substats'][1]
        self.critical = this_minion['cur_substats'][2]
        self.hit = this_minion['cur_substats'][3]
        self.hp = this_minion['cur_attributes'][0]
        self.mp = this_minion['mp']
        self.mp_capacity = this_minion['mp_capacity']
        self.mp_rate1 = this_minion['mp_regen_per_second']
        self.mp_rate2 = this_minion['mp_gain_rate_damage_dealt']
        self.mp_rate3 = this_minion['mp_gain_rate_damage_taken']
        self.luck = this_minion['luck']
        
        self.fmoves = {}
        self.smoves = {}
        
        for sid in this_minion['fmoves']:
            move = Skill(**skill_dict[sid])
            move.dps = move.power/(move.cast+move.cd)
            self.fmoves[move.id] = move
        
        smove_min_mp = 0
        for sid in this_minion['smoves']:
            move = Skill(**skill_dict[sid])
            move.mpdelta = np.ceil(move.mpdelta*self.luck[2]) #smove mp cost depends on luck
            move.dps = move.power/(move.cast+move.cd)
            move.dpm = move.dps/abs(move.mpdelta)
            self.smoves[move.id] = move
            if smove_min_mp < abs(move.mpdelta):
                smove_min_mp = abs(move.mpdelta)
        self.smove_min_mp = smove_min_mp
            
        


In [45]:
# pick minions, unserialize as object
mnn1 = BattleMinion(minion_dict=minion_data, skill_dict=skill_data, mid=1)
mnn2 = BattleMinion(minion_dict=minion_data, skill_dict=skill_data, mid=2)



In [ ]:
#tbc: team rule
class Party:
    def __init__(self, mnn_list=[]):
        self.lst = []
        #self.active_mnn = None
        for mnn in mnn_list:
            self.add(mnn)

    def __iter__(self):
        # So you don't have to write "for pkm in some_party.lst:"
        # Just "for pkm in some_party: " will do
        return iter(self.lst)

    def add(self, mnn):
        # Add pokemon to the party
        if len(self.lst) < MAX_MINION_PER_PARTY:
            self.lst.append(mnn)
            mnn.parent_party = self
            #if not self.active_mnn:
            #    self.active_mnn = mnn
        else:
            raise Exception("Failed to add new minion: Exceeding max party size")

    '''
    def next_mnn_up(self):
        # When active Pokemon (active_pkm) faints, this function is called
        # It puts the first alive Pokemon on the field

        for mnn in self:
            if mnn.hp > 0:
                self.active_mnn = mnn
                return
        raise StopIteration("Party exhausted")
    '''

    def alive(self):
        # If any of the minion in this party is still alive, returns True. 
        # Otherwise returns False
        for mnn in self:
            if mnn.hp > 0:
                return True
        return False

In [ ]:
MAX_MINION_PER_PARTY = 5
party1 = Party([mnn1])
party2 = Party([mnn2])


### battle classes

In [ ]:
class Event:
    '''
    events sit on the timeline and carry instructions / information about what they should change or do.
    
    TYPE OF EVENT
    mnnEnter: A minion has joined the battle! It can be atkr or dfdr
    mnnFree: Attacking minion is free to perform a fmove/smove, update current_move.free_since
                takes (name, t, mnn_usedAtk, current_move)
    mnnHurt: Attacking minion makes damage and gains mp (fmove) or loses mp (smove). 
                Defending minion takes damage and gains mp.
                takes (name, t, damage, mnn_usedAtk, mnn_hurt, atkr_mpdelta, dfdr_mpdelta, move_hurt_by)
    mpRegen: mp regeneration for all minions per second. update mp for all minions
                takes (name, t, mnn_usedAtk)
    announce: Announce a message to the log at a particular time. This is only used for 
               attack announcements that are not made at the same time they were decided.
               takes (name, t)
    backgroundDmg: The dfdr takes background dmg to emulate a team of attackers.
               takes (bgd_dmg_time, background_dmg_per_1000ms)
    '''
    
    def __init__(self, name, t, mnn_usedAtk=None, mnn_hurt=None, current_move=None, move_hurt_by=None
                 damage=None, atkr_mpdelta=None, dfdr_mpdelta=None, msg=None):
        # the name(type) of the event (dfdrHurt, atkrFree, etc) and the time it happens (s).
        self.name = name
        self.t = t
        self.mnn_usedAtk = mnn_usedAtk
        
        # for mnnHurt:
        self.mnn_hurt = mnn_hurt
        self.move_hurt_by = move_hurt_by
        self.damage = damage
        self.atkr_mpdelta = atkr_mpdelta # this may be negative
        self.dfdr_mpdelta = dfdr_mpdelta
        
        # for mnnFree:
        self.current_move = current_move        

    def __lt__(self, other): return self.t < other.t
    def __le__(self, other): return self.t <= other.t
    def __gt__(self, other): return self.t > other.t
    def __ge__(self, other): return self.t >= other.t
    def __eq__(self, other): return self.t == other.t
    

class Timeline:
    # A ordered queue to manage events
    
    def __init__(self):
        self.lst = []

    def __iter__(self):
        return iter(self.lst)

    def add(self, e):
        # add the event at the proper (sorted) spot.
        bisect.insort_left(self.lst, e)

    def pop(self):
        # return the first (earliest) event and remove it from the queue
        return self.lst.pop(0)

    def print(self):
        print("==Timeline== ", end="")
        for e in self.lst:
            print(str(e.t) + ":" + e.name, end=", ")
        print()

        
class World:
    # holds all settings
    
    self.mnn_data = None
    self.skill_data = None
    
    # team
    self.party1 = None
    self.party2 = None
    
    self.battle_type = None
    self.timelimit = 0
    self.tline = Timeline()
    
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            if hasattr(self, k):
                setattr(self, k, v)
    

In [ ]:
wd1_params = {
    'minion_data':minion_data,
    'skill_data': skill_data,
    'party1': [mnn1],
    'party2': [mnn2],
    'battle_type': '1v1',
    'timelimit': 600,
    'tline': Timeline()
}

wd1 = World(**wd1_params)



### battle functions

In [ ]:
def compute_damage(atkr, dfdr, move):

    def is_critical(critical_rate): # yes: 2 (critical hit creates 2x damage); no: 1
        return np.random.choice(np.arange(1,3), p=[1-critical_rate, critical_rate])
    
    def is_hit(hit_rate): # yes: 1; no: 0
        return np.random.choice(np.arange(0,2), p=[1-hit_rate, hit_rate])
    
    # compute damage
    # tbd: damage function
    damage = (np.floor(atkr.attack/dfdr.defense * is_critical(atkr.critical) * move.power) + 1) * is_hit(atkr.hit)
    dfdr.hp -= damage

    # compute mpdelta
    if move.mtype == 's':
        atkr_mpdelta = move.mpdelta
    else:
        atkr_mpdelta = np.ceil(damage*atkr.mp_gain_rate1)    
    
    dfdr_mpdelta = np.ceil(damage*dfdr.mp_gain_rate2)
    
    return damage, atkr_mpdelta, dfdr_mpdelta


In [ ]:
def mnn_use_move(atkr, dfdr, move, wd, t):
    #update mnn_dfdr.hp, mnn_atkr.mp, mnn_dfdr.mp, move.free_since
    #add events: mnnHurt, mnnFree
    
    damage, atkr_mpdelta, dfdr_mpdelta = compute_damage(atkr, dfdr, move)
    '''
    mnn_dfdr.hp -= damage
    mnn_dfdr.hp = max(mnn_dfdr.hp, 0)
    mnn_atkr.mp += atkr_mp_delta
    mnn_atkr.mp = min(mnn_atkr.mp, mnn_atkr.mp_capacity)
    mnn_dfdr.mp += dfdr_mp_delta
    mnn_dfdr.mp = min(mnn_dfdr.mp, mnn_dfdr.mp_capacity)
    move.free_since = t + move.cd
    '''
    #wd.tline.add(event("announce", t, mnn_usedAtk=mnn, move_hurt_by=move))
    wd.tline.add(Event("mnnHurt", t + move.cast, mnn_usedAtk=atkr, mnn_hurt=dfdr, damage=damage, atkr_mpdelta=atkr_mpdelta, dfdr_mpdelta=dfdr_mpdelta, move_hurt_by=move))
    wd.tline.add(Event("mnnFree", t + move.cd, mnn_usedAtk=atkr, current_move=move))
    
    
def regen_mp(party1, party2, wd, t):
    
    wd.tline.add(Event("mpRegen", t, ))
    

In [ ]:
def player_AI_choose(wd, atkr, t):
    # This function is the strategy of the attacking minion.
    # It directly manipulates the timeline object by inserting events.
    
    tline = wd.tline
    
    p1 = wd.party1
    p2 = wd.party2
    
    # select lowest hp minion in opponent team to attack
    dfdr_party = p2 if atkr.parent_party()==p1 else p1
    dfdr_min_hp = int('inf')
    for mnn in dfdr_party:
        if mnn.hp < dfdr_min_hp:
            dfdr = mnn
        
    def search_smoves(moveset, t, mp):
        available_smoves = {}
        for k,v in moveset.items():
            if mp + v.mpdelta >= 0 and v.free_since <= t:
                available_smoves[k] = v.dpm
        if len(available_smoves) >= 2:
            idx = max(available_smoves, key=available_smoves.get)
        elif len(available_smoves) == 1:
            idx = available_smoves.keys()[0]
        else:
            idx = None
        return idx
    
    
    def search_fmoves(moveset, t):
        available_fmoves = {}
        for k,v in moveset.items():
            if v.free_since <= t:
                available_fmoves[k] = v.dps
        if len(available_fmoves) >= 2:
            idx = max(available_fmoves, key=available_fmoves.get)
        elif len(available_fmoves) == 1:
            idx = available_fmoves.keys()[0]
        else:
            idx = None
        return idx
    
    
    if atkr.mp >= smove_min_mp:  # current mp meet min mp for smove, search smoves
        idx = search_smoves(atkr.smoves, t, atkr.mp)
        if idx != None:  # smove available, use smove
            mnn_use_move(atkr, dfdr, atkr.smoves[idx], wd, t)  
        else:  # no smove available, search fmoves
            idx = search_fmoves(atkr.fmoves, t)
            if idx != None:  # fmove available, use fmove
                mnn_use_move(atkr, dfdr, atkr.fmoves[idx], wd, t)
    else:  # current mp lower than min mp for smove, search fmoves
        idx = search_fmoves(atkr.fmoves, t)
        if idx != None:
            mnn_use_move(atkr, dfdr, atkr.fmoves[idx], wd, t)


In [ ]:
def init_atkr_event(party, wd): # free all moves
    tline = wd.tline
    
    for mnn in party:
        for k in mnn.fmoves.keys():
            tline.add(Event('mnnFree', 0, mnn_usedAtk=mnn, current_move=mnn.fmoves[k]))
        for k in mnn.smoves.keys():
            tline.add(Event('mnnFree', 0, mnn_usedAtk=mnn, current_move=mnn.smoves[k]))
            

In [ ]:
def battle(wd):
    tline = wd.tline
    p1 = wd.party1
    p2 = wd.party2
    
    init_atkr_event(p1, wd)
    init_atkr_event(p2, wd)
    
    while any([mnn.alive() for mnn in p1]) and any([mnn.alive() for mnn in p2]):
        this_event = tline.pop()
        t = this_event.t
        
        # case 1: a minion has a move freed
        if 'Free' in this_event.name:
            move_to_free = this_event.current_move
            move_to_free.free_since = t
            player_AI_choose(wd, this_event.mnn_usedAtk, t)
        
        # case 2: a minion takes damage and gains mp, check its available moves
        elif 'Hurt' in this_event.name:
            dmg_taker = this_event.mnn_hurt
            dmg_taker.hp -= this_event.damage
            dmg_taker.hp = max(dmg_taker.hp, 0)
            dmg_taker.mp = min(dmg_taker.mp + this_event.dfdr_mpdelta, dmg_taker.mp_capacity)
            
            dmg_giver = this_event.mnn_usedAtk
            dmg_giver.mp = min(dmg_giver.mp + this_event.atkr_mpdelta, dmg_giver.mp_capacity)
            this_event.move.free_since = t + this_event.move.cd
            
            player_AI_choose(wd, this_event.mnn_hurt, t)
            
            if this_event.move.mtype == 'f':
                player_AI_choose(wd, this_event.mnn_usedAtk, t)
    
    # battle finished. assign winner
    p1_alive = any([mnn.alive() for mnn in p1])
    p2_alive = any([mnn.alive() for mnn in p2])
    
    if p1_alive:
        winner = p1
        loser = p2
        print('Party 1 wins')
    else:
        winner = p2
        loser = p1
        print('Party 2 wins')